# full code

In [ ]:
# ------------------------------
# 1️⃣ โหลดไฟล์ train/test cleaned
# ------------------------------
import pandas as pd
import joblib

train_cleaned = pd.read_csv('/train_cleaned.csv')
test_cleaned = pd.read_csv('/test_cleaned.csv')

print("Train shape:", train_cleaned.shape)
print("Test shape:", test_cleaned.shape)

# ------------------------------
# 2️⃣ แยก Features / Target
# ------------------------------
y_train = train_cleaned['SalePrice']
X_train = train_cleaned.drop(columns=['Id', 'SalePrice'])
X_test = test_cleaned.drop(columns=['Id'])

# ------------------------------
# 3️⃣ One-hot encoding (ถ้ามี categorical)
# ------------------------------
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# ให้ columns ของ test match train
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)


# หลังจาก X_train = pd.get_dummies(X_train) เสร็จแล้ว
joblib.dump(X_train.columns, "feature_columns.pkl")


# ------------------------------
# 4️⃣ Train Linear Regression
# ------------------------------
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

# ------------------------------
# 5️⃣ ทำนาย Test Data และสร้าง submission
# ------------------------------
y_test_pred = model.predict(X_test)

submission = pd.DataFrame({
    'Id': test_cleaned['Id'],
    'SalePrice': y_test_pred
})
submission.to_csv('submission.csv', index=False)
print("Submission file created: submission.csv")
submission.head()

# ------------------------------
# 6️⃣ Validation RMSE & MAE (optional)
# ------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error



import numpy as np

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model.fit(X_tr, y_tr)
y_val_pred = model.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
mae = mean_absolute_error(y_val, y_val_pred)
mape = mean_absolute_percentage_error(y_val, y_val_pred)
print(f"Validation MAPE: {mape*100:.2f}%")
print("Validation RMSE:", rmse)
print("Validation MAE:", mae)

# ------------------------------
# 7️⃣ Interactive: ทำนายราคาบ้านจาก 3 feature หลัก
# ------------------------------
# ดูช่วงค่าของ feature หลัก ๆ
print("\nช่วงค่าของ feature หลัก ๆ:")
print(train_cleaned[['LotArea','OverallQual','YearBuilt']].describe())

# ให้ผู้ใช้กรอกค่า
lot_area_input = float(input("กรุณากรอกขนาดพื้นที่บ้าน (LotArea, ft²) [ตัวอย่าง 8000]: "))
overall_qual_input = int(input("กรุณากรอกคุณภาพบ้าน (OverallQual, 1-10) [ตัวอย่าง 7]: "))
year_built_input = int(input("กรุณากรอกปีสร้างบ้าน (YearBuilt) [ตัวอย่าง 2005]: "))

# สร้าง DataFrame สำหรับทำนาย
X_new = pd.DataFrame(0, index=[0], columns=X_train.columns)
X_new['LotArea'] = lot_area_input
X_new['OverallQual'] = overall_qual_input
X_new['YearBuilt'] = year_built_input

# ทำนายราคาบ้าน
pred_price = model.predict(X_new)
print(f"\nราคาบ้านที่ทำนาย: {pred_price[0]:,.2f} บาท")

import joblib
joblib.dump(model, "model.pkl")   # เซฟ
model = joblib.load("model.pkl")  # โหลดกลับมาใช้

Train shape: (1460, 78)
Test shape: (1459, 77)
Submission file created: submission.csv
Validation RMSE: 29647.750843021928
Validation MAE: 18467.044741419406

ช่วงค่าของ feature หลัก ๆ:
             LotArea  OverallQual    YearBuilt
count    1460.000000  1460.000000  1460.000000
mean    10516.828082     6.099315  1971.267808
std      9981.264932     1.382997    30.202904
min      1300.000000     1.000000  1872.000000
25%      7553.500000     5.000000  1954.000000
50%      9478.500000     6.000000  1973.000000
75%     11601.500000     7.000000  2000.000000
max    215245.000000    10.000000  2010.000000
กรุณากรอกขนาดพื้นที่บ้าน (LotArea, ft²) [ตัวอย่าง 8000]: 15000
กรุณากรอกคุณภาพบ้าน (OverallQual, 1-10) [ตัวอย่าง 7]: 9
กรุณากรอกปีสร้างบ้าน (YearBuilt) [ตัวอย่าง 2005]: 2005

ราคาบ้านที่ทำนาย: 26,956.89 บาท
